In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
pydrake.common.set_log_level("warn")

'info'

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
#add iiwa
iiwa = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
    pydrake.common.FindResourceOrThrow(
        "drake/examples/push_box/iiwa7/iiwa7_with_box_collision.sdf"), "iiwa"
)

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))


# Set default positions:
q0 = [0.0, 0.1, 0, -1.2, 0, 1.6, 0]
index = 0
for joint_index in plant.GetJointIndices(iiwa):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, pydrake.multibody.tree.RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1


In [3]:
#add gripper
cylinder_gripper = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/cylinder_gripper.sdf"), "cylinder_gripper")

X_7G = RigidTransform(RotationMatrix(), [0, 0, 0.033])
# X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa),
                 plant.GetFrameByName("base_link", cylinder_gripper), X_7G)

<WeldJoint_[float] name='iiwa_link_7_welds_to_base_link' index=8 model_instance=3>

In [4]:
#add table
table = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/extra_heavy_duty_table_surface_only_collision_lower_height.sdf"), "table")

X_WT = RigidTransform(RotationMatrix(), [0.7, 0., 0.])
plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("link", table), X_WT)


#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box.sdf"), "cracker_box")
#add box_goal
cracker_box_goal = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_goal.sdf"), "cracker_box_goal")

In [5]:
plant.Finalize()
plant.set_name("plant")

In [6]:
#add InverseDynamicsController
num_iiwa_positions = plant.num_positions(iiwa)

# I need a PassThrough system so that I can export the input port.
iiwa_position = builder.AddSystem(pydrake.systems.primitives.PassThrough(num_iiwa_positions))
# builder.ExportInput(iiwa_position.get_input_port(), "iiwa_position")
builder.ExportOutput(iiwa_position.get_output_port(), "iiwa_position_command")

# Export the iiwa "state" outputs.
demux = builder.AddSystem(pydrake.systems.primitives.Demultiplexer(
    2 * num_iiwa_positions, num_iiwa_positions))
builder.Connect(plant.get_state_output_port(iiwa), demux.get_input_port())
builder.ExportOutput(demux.get_output_port(0), "iiwa_position_measured")
builder.ExportOutput(demux.get_output_port(1), "iiwa_velocity_estimated")
builder.ExportOutput(plant.get_state_output_port(iiwa), "iiwa_state_estimated")

# Make the plant for the iiwa controller to use.
controller_plant = pydrake.multibody.plant.MultibodyPlant(time_step=1e-4)
controller_iiwa =  pydrake.multibody.parsing.Parser(controller_plant).AddModelFromFile(
    pydrake.common.FindResourceOrThrow(
        "drake/examples/push_box/iiwa7/iiwa7_with_box_collision.sdf"), "controller_iiwa"
)
controller_plant.WeldFrames(controller_plant.world_frame(), controller_plant.GetFrameByName("iiwa_link_0"))
controller_plant.Finalize()


# Add the iiwa controller
iiwa_controller = builder.AddSystem(
    pydrake.systems.controllers.InverseDynamicsController(
        controller_plant,
        kp=[100]*num_iiwa_positions,
        ki=[1]*num_iiwa_positions,
        kd=[20]*num_iiwa_positions,
        has_reference_acceleration=False))
iiwa_controller.set_name("iiwa_controller")
builder.Connect(
    plant.get_state_output_port(iiwa), iiwa_controller.get_input_port_estimated_state())

# Add in the feed-forward torque
adder = builder.AddSystem(pydrake.systems.primitives.Adder(2, num_iiwa_positions))
builder.Connect(iiwa_controller.get_output_port_control(),
                adder.get_input_port(0))
# Use a PassThrough to make the port optional (it will provide zero values if not connected).
torque_passthrough = builder.AddSystem(
    pydrake.systems.primitives.PassThrough([0]*num_iiwa_positions))
builder.Connect(torque_passthrough.get_output_port(), adder.get_input_port(1))
# builder.ExportInput(torque_passthrough.get_input_port(), "iiwa_feedforward_torque")
builder.Connect(adder.get_output_port(), plant.get_actuation_input_port(iiwa))

# Add discrete derivative to command velocities.
desired_state_from_position = builder.AddSystem(
    pydrake.systems.primitives.StateInterpolatorWithDiscreteDerivative(
        num_iiwa_positions, time_step=1e-4, suppress_initial_transient=True))
desired_state_from_position.set_name("desired_state_from_position")
builder.Connect(desired_state_from_position.get_output_port(),      
                iiwa_controller.get_input_port_desired_state())
builder.Connect(iiwa_position.get_output_port(), desired_state_from_position.get_input_port())

# Export commanded torques.
#builder.ExportOutput(adder.get_output_port(), "iiwa_torque_commanded")
#builder.ExportOutput(adder.get_output_port(), "iiwa_torque_measured")

# Export "cheat" ports.
builder.ExportOutput(scene_graph.get_query_output_port(), "geometry_query")
builder.ExportOutput(plant.get_contact_results_output_port(), "contact_results")
builder.ExportOutput(plant.get_state_output_port(), "plant_continuous_state")

OutputPortIndex(6)

In [7]:
robot = controller_plant
params = pydrake.manipulation.planner.DifferentialInverseKinematicsParameters(
    robot.num_positions(), robot.num_velocities())


params.set_timestep(1e-4)
# True velocity limits for the IIWA14 (in rad, rounded down to the first
# decimal)
iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
differential_ik = builder.AddSystem(
    pydrake.manipulation.planner.DifferentialInverseKinematicsIntegrator(
        robot, robot.GetFrameByName("iiwa_link_7"), 1e-4, params))
builder.Connect(differential_ik.get_output_port(),
                iiwa_position.get_input_port())

teleop = builder.AddSystem(PoseSliders(
    min_range = PoseSliders.MinRange(roll=0, pitch=-0.5, yaw=-np.pi, 
                                     x=-0.6, y=-0.8, z=0.0),
    max_range = PoseSliders.MaxRange(roll=2*np.pi, pitch=np.pi, yaw=np.pi,
                                     x=0.8, y=0.3, z=1.1)
))
builder.Connect(teleop.get_output_port(0), 
                differential_ik.get_input_port())

FloatSlider(value=0.0, description='roll', layout=Layout(width='90%'), max=6.283185307179586, step=0.01)

FloatSlider(value=0.0, description='pitch', layout=Layout(width='90%'), max=3.141592653589793, min=-0.5, step=…

FloatSlider(value=0.0, description='yaw', layout=Layout(width='90%'), max=3.141592653589793, min=-3.1415926535…

FloatSlider(value=0.0, description='x', layout=Layout(width='90%'), max=0.8, min=-0.6, step=0.01)

FloatSlider(value=0.0, description='y', layout=Layout(width='90%'), max=0.3, min=-0.8, step=0.01)

FloatSlider(value=0.0, description='z', layout=Layout(width='90%'), max=1.1, step=0.01)

In [8]:
iiwa_link_7_body = plant.GetBodyByName("iiwa_link_7", iiwa)
iiwa_link_7_id = plant.GetBodyFrameIdIfExists(iiwa_link_7_body.index())
cylinder_gripper_body = plant.GetBodyByName("base_link", cylinder_gripper)
cylinder_gripper_id = plant.GetBodyFrameIdIfExists(cylinder_gripper_body.index())
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
table_link_body = plant.GetBodyByName("link", table)
table_link_id = plant.GetBodyFrameIdIfExists(table_link_body.index())

draw_frames = True
frames_to_draw = [iiwa_link_7_id, cylinder_gripper_id, cracker_box_id, table_link_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
X_WB = RigidTransform(RotationMatrix(), [0.5, 0., 0.3285 + 0.03295])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box),
                              X_WB)

X_WBG = RigidTransform(RotationMatrix(), [0.8, 0., 0.3285 + 0.03295])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box_goal),
                              X_WBG)

meshcat.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


In [9]:
simulator = pydrake.systems.analysis.Simulator(diagram)
# context = simulator.get_mutable_context()

q0 = plant.GetPositions(plant_context, iiwa)
print(q0)
differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
differential_ik.SetPositions(diff_ik_context, q0)
teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

[ 0.   0.1  0.  -1.2  0.   1.6  0. ]


In [ ]:
if running_as_notebook:  # Then we're not just running as a test on CI.
    simulator.set_target_realtime_rate(1.0)

    stop_button = ToggleButton(value=False, description='Stop Simulation')
    display(stop_button)
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
    stop_button.value = False
  
else:
    simulator.AdvanceTo(0.1)

ToggleButton(value=False, description='Stop Simulation')

[2021-11-22 22:19:25.878] [console] [warning] Differential IK started returning status:"stuck" at time 0.0.


In [11]:
print(q0)

[ 0.   0.1  0.  -1.2  0.   1.6  0. ]
